In [9]:
##modules
#%matplotlib widget
#%matplotlib inline
#
#%matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('TkAgg')  # Asegúrate de que este backend está instalado.

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

# aplicar la acf EN epochs

from statsmodels.tsa.stattools import acf
import numpy as np

import pandas as pd

In [6]:
# Variables path
layer_script = "block"

#process=""
disco="g"

subj = "S01"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\\PROYECTO_SELF")

# #carpetas generales de datos
brain_vision_dir = datadir /"Data_self"/"Export_acw_Brain_Vision"
fif_data = datadir /"Data_self"/ "fif_data" 


#carpeta analysis
output_analysis= datadir / "output_analysis"

acw_analysis_path = output_analysis / "acw"
acw_analysis_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de preprocesado
# output_preproc = datadir / "output_preproc"

# preproc_path = output_preproc / f"preproc_{layer_script}"
# preproc_path.mkdir(parents=True, exist_ok=True)
 
# # Carpeta de epocas "sucias"
# epochs_path = preproc_path / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True) 

# # Carpeta de ICA
# ICA_path = preproc_path / f"ICA_{layer_script}"
# ICA_path.mkdir(parents=True, exist_ok=True)

# # Épocas limpias
# epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
# epochs_clean_path.mkdir(parents=True, exist_ok=True)

# #epocas epochs
# epochs_path = Path(preproc_path) / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True)
 

# # Definir la carpeta de output_source antes de usarla
# output_source = Path(r"g:\MOUS_204\output_source")

# source_path = output_source / f"source_{layer_script}"
# source_path.mkdir(parents=True, exist_ok=True)

# #raw_hsp es el raw con fiducials cargados
# raw_hsp_path = source_path / f"raw_hsp"
# raw_hsp_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de forward solution
# fwd_path = source_path / f"fwd"
# fwd_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de inverse solution
# inverse_path = source_path / f"inverse"
# inverse_path.mkdir(parents=True, exist_ok=True)


# mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)


In [11]:

#epochs
combinaciones = ["S1 - S4","S1 - S5", "S2 - S4", "S2 - S5", "S3 - S4", "S3 - S5"]

combinacion= combinaciones[1]

# Obtener todos los archivos y carpetas dentro de la ruta
subjects = list(fif_data.iterdir())

# Filtrar solo los nombres de sujetos (removiendo "_epochs")
subject_names = [subject.stem.replace("_epochs", "") for subject in subjects]

subj=subject_names[0]
subj


# # epochs_zinnen_path = epochs_clean_path / f"{subj}_epochs_zinnen_{type_script}"
# # epochs_path = epochs_clean_path / f"{subj}_epochs_{type_script}"

path_epochs= fif_data / f"{subj}_epochs" / f"{subj}_{combinacion}-epo.fif"

epochs = mne.read_epochs(path_epochs) 





Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated


In [ ]:
# aplicar la acf EN epochs

from statsmodels.tsa.stattools import acf
import numpy as np

import pandas as pd

def acf_epochs(epochs,adjusted=False,fft=True,alpha=None, bartlett_confint=True, missing="none", isplot=False): 
    
    #import epochs object and get data ONLY ON MEG DATA
    data_epochs=epochs.copy().pick("eeg").get_data()

    #get duration
    duration= epochs.tmax - epochs.tmin
    #get sample frequency
    sfreq= epochs.info['sfreq']
    #get lags
    lags=duration*sfreq


    #list to store the values of acf, acw_50,acw_0 for each epoch for each sensor for each EPOCH
    acf_elect_all_epoch_all = []
    acf_mean_elect_all_epoch_all=[]

    #conjunto de todos los valores de ACW_0-ACW_50 para cada sensor en cada epoca
    acw_50_elect_all_epoch_all = []
    acw_0_elect_all_epoch_all = []


    #for each epoch
      
    for j in range(0,len(data_epochs)):
        #take data for each epoch
        data_epoch=data_epochs[j]

        #list to store the values of acf, acw_50,acw_0 for each SENSOR for each EPOCH

        acf_elect_all_epoch = []
        
        acf_mean_elect_all_epoch = []

        acw_50_elect_all_epoch = []
        acw_0_elect_all_epoch = []

    


        #calculus of value  for EACH SENSOR
        for i in range(0,len(data_epoch)):
            #lags se puede dejar por defecto porque te cogelos valores
            #hasta los que tiene sentido calcularlo, aunque yo voy a calcularlo para todos los lags
            #calculo de acf de sensor
            #alfa lo dejo endefault que es la confianza del 95% 

            acf_elect_epoch, qstat_vals_elect_epoch, pvals_elect_epoch = acf(
                data_epoch[i],
                adjusted=adjusted,
                fft=fft,
                qstat=True,
                nlags=lags,
                alpha=alpha,
                bartlett_confint=bartlett_confint,
                missing=missing
            )

            #conjunto de valores de acf en un electrodo en una epoca
            acf_elect_all_epoch.append(acf_elect_epoch)

            #promedio de valores de acf en un electrodo en una epoca
            acf_mean_elect_all_epoch.append(np.mean(acf_elect_epoch))


            #calculo de acw_50 y acw_0 de sensor for both in lags and in seconds
            #recuerda que acw_50 y acw_0 son valores unicos, no como acf que es un conjunto de valores
            acw_50_lags_elect_epoch = np.argmax(acf_elect_epoch <= 0.5)
            acw_50_s_elect_epoch = acw_50_lags_elect_epoch / sfreq

            acw_50_elect_all_epoch.append(acw_50_s_elect_epoch)


            acw_0_lags_elect_epoch= np.argmax(acf_elect_epoch <= 0)
            acw_0_s_elect_epoch = acw_0_lags_elect_epoch / sfreq

            acw_0_elect_all_epoch.append(acw_0_s_elect_epoch)


        #APPEND valores de TODOS LOS ELECTODOS en TODAS LAS EPOCAS

        acf_elect_all_epoch_all.append(acf_elect_all_epoch)
        acf_mean_elect_all_epoch_all.append(acf_mean_elect_all_epoch)

        acw_50_elect_all_epoch_all.append(acw_50_elect_all_epoch)
        acw_0_elect_all_epoch_all.append(acw_0_elect_all_epoch)


       

    #promedio de valores de acw_50 y acw_0 en un electrodo para todas las epocas
    # acw_50_elect_all_epoch_mean = []
    # acw_0_elect_all_epoch_mean = []


    #promedio de los electrodos en todas las epocas
    # Convertir la lista de listas en un array de NumPy para cálculos más eficientes
    acw_50_elect_all_epoch_all_array = np.array(acw_50_elect_all_epoch_all)  # (21 filas, 59 columnas)
    acw_0_elect_all_epoch_all_array = np.array(acw_0_elect_all_epoch_all)  # (21 filas, 59 columnas)

    ##computo de la media de TODOS los electrodos en CADA EPOCA
    acw_50_elect_mean_epoch_all = np.mean(acw_50_elect_all_epoch_all_array, axis=1)
    acw_0_elect_mean_epoch_all = np.mean(acw_0_elect_all_epoch_all_array, axis=1)


    ##computo de la media de cada los electrodos en TODAS EPOCAS
    acw_50_elect_all_epoch_mean = np.mean(acw_50_elect_all_epoch_all_array, axis=0)       
    acw_0_elect_all_epoch_mean = np.mean(acw_0_elect_all_epoch_all_array, axis=0)
    

    # # Crear un DataFrame con las columnas necesarias

    table_autocorrelation = pd.DataFrame({
        'Subject': [subj] * len(acf_elect_all_epoch_all),  # Repite el sujeto para todas las filas
        'Combination': [combinacion] * len(acf_elect_all_epoch_all),  # Repite la combinación para todas las filas
        "Epoch": range(1, len(acf_elect_all_epoch_all) + 1),  # Número de épocas
        'acf_elect_all_epoch_all': acf_elect_all_epoch_all,
        "acf_mean_elect_all_epoch_all": acf_mean_elect_all_epoch_all,
        'acw_50_elect_all_epoch_all': acw_50_elect_all_epoch_all,
        'acw_0_elect_all_epoch_all': acw_0_elect_all_epoch_all,
        'acw_50_elect_mean_epoch_all': [acw_50_elect_mean_epoch_all] * len(acf_elect_all_epoch_all),
        'acw_0_elect_mean_epoch_all': [acw_0_elect_mean_epoch_all] * len(acf_elect_all_epoch_all),
        "acw_50_elect_all_epoch_mean": [acw_50_elect_all_epoch_mean] * len(acf_elect_all_epoch_all),
        "acw_0_elect_all_epoch_mean": [acw_0_elect_all_epoch_mean] * len(acf_elect_all_epoch_all)
        })

    #identificar acw_50


    #valores de estadística
        ## pero ver que valores son puto significativos de la ACF o de la ACW??



    isplot=False    

    # if isplot==True:
    #     # 1) Creamos un vector de lags en segundos,
    #     #    asumiendo que acf_val tiene tantos puntos como lags + 1.
    #     time_lags = np.arange(len(acf_val)) / sfreq

    #     # 2) Convertimos los índices (entero) de ACW_50 y ACW_0
    #     ACW_50_i = int(acw_50_lags)
    #     ACW_0_i = int(acw_0_lags)

    #     # 3) Figura
    #     plt.figure(figsize=(8, 4))

    #     # 4) Dibujamos la ACF en negro
    #     plt.plot(time_lags, acf_val, 'k', label='ACF')

    #     # 5) Acotamos el eje X a la duración total de la señal (o a lo que consideres)
    #     plt.xlim([0, time_lags[-1]])

    #     # 6) Rellenamos el área hasta ACW_50
    #     plt.fill_between(
    #         time_lags[:ACW_50_i + 1],
    #         acf_val[:ACW_50_i + 1],
    #         color='r', alpha=0.3, label='ACW-50 area'
    #     )

    #     # 7) Rellenamos el área hasta ACW_0
    #     plt.fill_between(
    #         time_lags[:ACW_0_i + 1],
    #         acf_val[:ACW_0_i + 1],
    #         color='m', alpha=0.3, label='ACW-0 area'
    #     )

    #     # 8) Título con valores de ACW
    #     plt.title(f'ACW-0 = {acw_0:.1f} s    ACW-50 = {acw_50:.1f} s')
    #     plt.xlabel('Lags (s)')
    #     plt.ylabel('Autocorrelation')
    #     plt.legend(loc='best')
    #     plt.show()

    return table_autocorrelation
    # return acf_epoch_all,acw_50_elect_all_epoch_all, acw_0_elect_all_epoch_all




In [45]:
table_autocorrelation= acf_epochs(epochs, isplot=False)

C:\Users\UCM\AppData\Local\Temp\ipykernel_13384\2617573886.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


In [178]:
##codigo para agrupar todas las tablas

all_tables = []

for i in range(0,len(subject_names)):
    for h in range(0,len(combinaciones)):
        subj=subject_names[i]
        combinacion= combinaciones[h]
        path_epochs= fif_data / f"{subj}_epochs" / f"{subj}_{combinacion}-epo.fif"
        epochs = mne.read_epochs(path_epochs)
        table_autocorrelation= acf_epochs(epochs, isplot=False)
        all_tables.append(table_autocorrelation)

final_table = pd.concat(all_tables, ignore_index=True)


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S4-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
19 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S2 - S4-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
17 matching events found
No baseline correction applied
0 projection items activated


...

C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S06_epochs\S06_S3 - S4-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S06_epochs\S06_S3 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
24 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


In [51]:
table_autocorrelation

,Subject,Combination,Epoch,acf_elect_all_epoch_all,acf_mean_elect_all_epoch_all,acw_50_elect_all_epoch_all,acw_0_elect_all_epoch_all,acw_50_elect_mean_epoch_all,acw_0_elect_mean_epoch_all,acw_50_elect_all_epoch_mean,acw_0_elect_all_epoch_mean
0,S01,S1 - S5,1,"[[1.0, 0.9690277709544161, 0.8957519557069537,...","[0.0001780084495609426, 0.00821106594098748, 0...","[0.022, 0.02, 0.02, 0.036, 0.058, 0.034, 0.022...","[0.522, 0.684, 0.506, 0.898, 1.762, 1.13, 0.49...",0.043220,1.263220,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
1,S01,S1 - S5,2,"[[1.0, 0.9742031524378932, 0.9124558845694722,...","[0.014935298078953317, 0.00654349313843797, 0....","[0.038, 0.026, 0.032, 0.038, 0.076, 0.05, 0.02...","[0.48, 0.446, 0.428, 0.536, 0.49, 0.448, 0.448...",0.037051,0.513932,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
2,S01,S1 - S5,3,"[[1.0, 0.979652656049055, 0.9324948998089536, ...","[-0.09623095061179981, -0.16804014772548523, -...","[0.032, 0.034, 0.048, 0.044, 0.036, 0.026, 0.0...","[1.882, 2.742, 2.9, 1.87, 1.022, 0.226, 1.006,...",0.031220,0.602508,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
3,S01,S1 - S5,4,"[[1.0, 0.9794573414075601, 0.9311543733016583,...","[-0.0913324152513611, -0.1775039821343573, -0....","[0.036, 0.054, 0.054, 0.034, 0.174, 0.05, 0.05...","[2.032, 2.156, 2.146, 0.732, 2.74, 0.74, 0.724...",0.036373,0.732949,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
4,S01,S1 - S5,5,"[[1.0, 0.9868705645439952, 0.9554482488991816,...","[0.02876746779799747, 0.04137258949636905, 0.0...","[0.028, 0.03, 0.03, 0.03, 0.034, 0.036, 0.034,...","[0.608, 0.606, 0.614, 0.442, 0.772, 0.764, 0.5...",0.033559,0.868034,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
5,S01,S1 - S5,6,"[[1.0, 0.9899325168995756, 0.9647891923301427,...","[-0.00969529794346642, -0.04426935303450281, -...","[0.032, 0.036, 0.036, 0.03, 0.084, 0.036, 0.03...","[0.9, 0.88, 0.75, 0.906, 1.246, 0.738, 0.638, ...",0.042034,0.629525,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
6,S01,S1 - S5,7,"[[1.0, 0.9899504817494202, 0.963824889698331, ...","[0.0012839468294756782, -0.026036064628658952,...","[0.026, 0.026, 0.034, 0.032, 0.028, 0.03, 0.03...","[0.308, 0.332, 2.01, 0.382, 0.232, 0.316, 2.06...",0.032881,0.680441,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
7,S01,S1 - S5,8,"[[1.0, 0.9922634815789326, 0.9723448312716317,...","[-0.04133638076531709, 0.009998137272575468, -...","[0.038, 0.032, 0.04, 0.036, 0.042, 0.034, 0.03...","[0.74, 0.598, 0.76, 0.582, 0.748, 0.224, 0.218...",0.030610,0.354136,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
8,S01,S1 - S5,9,"[[1.0, 0.986460276357467, 0.954339660419086, 0...","[0.02507036107916501, -0.02939978298175851, -0...","[0.04, 0.032, 0.03, 0.042, 0.056, 0.038, 0.028...","[0.702, 0.672, 0.802, 0.66, 0.998, 0.822, 0.81...",0.033220,0.819661,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."
9,S01,S1 - S5,10,"[[1.0, 0.9808252787305254, 0.9359828717222002,...","[-0.05345975225600691, -0.10263428296877271, -...","[0.028, 0.034, 0.036, 0.024, 0.024, 0.03, 0.02...","[2.364, 2.38, 1.97, 0.858, 0.474, 1.138, 0.786...",0.028644,0.777186,"[0.038666666666666676, 0.033142857142857154, 0...","[0.9984761904761905, 1.0647619047619048, 1.219..."


In [ ]:
final_table.to_csv(acw_analysis_path / f"final_table_acw_1-6.csv", index=False)

In [ ]:
##code to save the dataframe 

table_autocorrelation.to_csv(analysis_path / f{subj}_table_autocorrelation.csv", index=False)

In [3]:
import pandas as pd

In [4]:
# code to read the dataframe

final_table = pd.read_csv(analysis_path / f"final_table_acw_1-6.csv")


NameError: name 'analysis_path' is not defined

In [186]:
df_acf_loaded

,Subject,Sensor,ACF_Zinnen,ACW_50_Zinnen,ACW_0_Zinnen,ACF_Woorden,ACW_50_Woorden,ACW_0_Woorden
0,sub-A2004,1,[ 1.00000000e+00 9.90504844e-01 9.62689459e-...,0.016667,0.395000,[ 1. 0.98957351 0.95919365 ... -1.02...,0.016667,0.343333
1,sub-A2004,2,[ 1. 0.99110624 0.96506121 ... -0.07...,0.018333,0.361667,[ 1. 0.99033796 0.96210035 ... 0.00...,0.016667,0.351667
2,sub-A2004,3,[ 1. 0.9913972 0.9661997 ... -0.21...,0.018333,0.398333,[ 1. 0.99071471 0.96354885 ... -0.08...,0.018333,0.351667
3,sub-A2004,4,[ 1. 0.99097473 0.96453989 ... -0.14...,0.016667,0.471667,[1. 0.99063727 0.96321899 ... 0.081492...,0.016667,0.328333
4,sub-A2004,5,[ 1. 0.99162543 0.96704988 ... -0.01...,0.018333,0.468333,[1. 0.99126763 0.96565904 ... 0.058078...,0.018333,0.266667
...,...,...,...,...,...,...,...,...
268,sub-A2004,269,[ 1. 0.98899351 0.95682109 ... -0.01...,0.015000,0.351667,[ 1. 0.98725621 0.95016538 ... -1.19...,0.013333,0.263333
269,sub-A2004,270,[ 1. 0.99171923 0.96734816 ... -0.01...,0.016667,0.041667,[1. 0.99178445 0.96756315 ... 0.375049...,0.016667,0.033333
270,sub-A2004,271,[ 1. 0.99140437 0.96611658 ... -0.32...,0.016667,0.035000,[1. 0.99110044 0.96495064 ... 0.241846...,0.016667,0.033333
271,sub-A2004,272,[1. 0.98998846 0.96066231 ... 0.008007...,0.016667,0.346667,[1. 0.98951551 0.95888772 ... 0.563044...,0.015000,0.041667


In [14]:
#in case you want to add a new subjet to the dataframe its better to concatenate it
# 
epochs 

Number of events,21
Events,1: 21
Time range,0.000 – 9.198 s
Baseline,off


In [20]:
#import epochs object and get data ONLY ON MEG DATA
data_epochs=epochs.copy().pick("eeg").get_data()

#get duration
duration= epochs.tmax - epochs.tmin
#get sample frequency
sfreq= epochs.info['sfreq']
#get lags
lags=duration*sfreq

#del epochs

#list to store the values of acf, acw_50,acw_0 for each epoch for each sensor for each EPOCH
acf_elect_all_epoch_all = []
acf_mean_elect_all_epoch_all=[]

#conjunto de todos los valores de ACW_0-ACW_50 para cada sensor en cada epoca
acw_50_elect_all_epoch_all = []
acw_0_elect_all_epoch_all = []

#promedio de valores de acw_50 y acw_0 para cada epoca (el promedio de todos los sensores en una epoca)
acw_50_elect_mean_epoch_all = []
acw_0_elect_mean_epoch_all = []







#for each epoch
    
for j in range(0,len(data_epochs)):
    #take data for each epoch
    data_epoch=data_epochs[j]

    #list to store the values of acf, acw_50,acw_0 for each SENSOR for each EPOCH

    acf_elect_all_epoch = []
    acf_mean_elect_all_epoch = []

    acw_50_elect_all_epoch = []
    acw_0_elect_all_epoch = []




    #calculus of value  for EACH SENSOR
    for i in range(0,len(data_epoch)):
        #lags se puede dejar por defecto porque te cogelos valores
        #hasta los que tiene sentido calcularlo, aunque yo voy a calcularlo para todos los lags
        #calculo de acf de sensor
        #alfa lo dejo endefault que es la confianza del 95% 

        acf_elect_epoch, qstat_vals_elect_epoch, pvals_elect_epoch = acf(
            data_epoch[i],
            adjusted=True,
            fft=True,
            qstat=True,
            nlags=lags
        )

        #conjunto de valores de acf en un electrodo en una epoca
        acf_elect_all_epoch.append(acf_elect_epoch)

        #promedio de valores de acf en un electrodo en una epoca
        acf_mean_elect_all_epoch.append(np.mean(acf_elect_epoch))


        #calculo de acw_50 y acw_0 de sensor for both in lags and in seconds
        #recuerda que acw_50 y acw_0 son valores unicos, no como acf que es un conjunto de valores
        acw_50_lags_elect_epoch = np.argmax(acf_elect_epoch <= 0.5)
        acw_50_s_elect_epoch = acw_50_lags_elect_epoch / sfreq

        acw_50_elect_all_epoch.append(acw_50_s_elect_epoch)


        acw_0_lags_elect_epoch= np.argmax(acf_elect_epoch <= 0)
        acw_0_s_elect_epoch = acw_0_lags_elect_epoch / sfreq

        acw_0_elect_all_epoch.append(acw_0_s_elect_epoch)


    #APPEND valores de TODOS LOS ELECTODOS en TODAS LAS EPOCAS

    acf_elect_all_epoch_all.append(acf_elect_all_epoch)
    acf_mean_elect_all_epoch_all.append(acf_mean_elect_all_epoch)

    acw_50_elect_all_epoch_all.append(acw_50_elect_all_epoch)
    acw_0_elect_all_epoch_all.append(acw_0_elect_all_epoch)

    #promedio de ACW de los electrodos en una epoca
    acw_50_elect_mean_epoch=np.mean(acw_50_elect_all_epoch)
    acw_0_elect_mean_epoch=np.mean(acw_0_elect_all_epoch)

    #APPEND de promedio de los electrodos en una epoca, osea el promedio de acw en todoso los electrodos en una epoca
    acw_50_elect_mean_epoch_all.append(acw_50_elect_mean_epoch)
    acw_0_elect_mean_epoch_all.append(acw_0_elect_mean_epoch)
    


C:\Users\UCM\AppData\Local\Temp\ipykernel_13384\2126086916.py:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


In [21]:
acw_50_elect_mean_epoch_all

[np.float64(0.043220338983050846),
 np.float64(0.037050847457627115),
 np.float64(0.03122033898305085),
 np.float64(0.036372881355932196),
 np.float64(0.03355932203389831),
 np.float64(0.04203389830508474),
 np.float64(0.03288135593220339),
 np.float64(0.030610169491525424),
 np.float64(0.033220338983050844),
 np.float64(0.028644067796610172),
 np.float64(0.028372881355932206),
 np.float64(0.02850847457627119),
 np.float64(0.034338983050847455),
 np.float64(0.04779661016949152),
 np.float64(0.026033898305084745),
 np.float64(0.034474576271186434),
 np.float64(0.08233898305084746),
 np.float64(0.02959322033898305),
 np.float64(0.0800677966101695),
 np.float64(0.029559322033898303),
 np.float64(0.054203389830508465)]

In [ ]:

#promedio de valores de acw_50 y acw_0 en un electrodo para todas las epocas
# acw_50_elect_all_epoch_mean = []
# acw_0_elect_all_epoch_mean = []

#promedio de los electrodos en todas las epocas
# Convertir la lista de listas en un array de NumPy para cálculos más eficientes
acw_50_elect_all_epoch_all_array = np.array(acw_50_elect_all_epoch_all)  # (21 filas, 59 columnas)
# Calcular el promedio a lo largo de las columnas (axis=0) → Promedio por electrodo en cada época
acw_50_elect_all_epoch_mean = np.mean(acw_50_elect_all_epoch_all_array, axis=0)

acw_0_epoch_all_array = np.array(acw_0_elect_all_epoch_all)  # (21 filas, 59 columnas)
# Calcular el promedio a lo largo de las columnas (axis=0) → Promedio por electrodo en cada época       
acw_0_elect_all_epoch_mean = np.mean(acw_0_epoch_all_array, axis=0)

# #transformation of list to array
# acf_elect_all=np.array(acf_elect_all)
# acw_50_elect_all_s=np.array(acw_50_elect_all_s)
# acw_0_elect_all_s=np.array(acw_0_elect_all_s)

